In [4]:
# -*- coding: utf-8 -*-
import pygame
import math
import random

# --- ค่าคงที่ของเกม (จำเป็นต้องกำหนด) ---
WIDTH, HEIGHT = 800, 600
BLOCK_SIZE = 20  # ขนาดของบล็อกต่อตัวอักษร
BACKGROUND_COLOR = (10, 20, 40)  # สีพื้นหลัง (น้ำเงินเข้ม)
WATER_COLOR = (20, 80, 200, 200)  # สีน้ำ (มีค่า alpha)
SURFACE_HIGHLIGHT = (100, 150, 255, 100) # สีคลื่นผิวน้ำ

# --- คลาสที่ 1: typingText (แก้ไขให้สมบูรณ์) ---
class typingText:
    def __init__(self, screen, font, question, valid_answers):
        self.screen = screen
        self.font = font
        self.question = question
        # สร้าง set ของคำตอบ (ตัวพิมพ์เล็ก) เพื่อให้ค้นหาได้เร็ว
        self.valid_answers = set(ans.lower() for ans in valid_answers)
        self.user_input = ""
        self.message = ""
        self.message_color = (255, 255, 255)

    def handle_event(self, event):
        """
        จัดการการพิมพ์ของผู้เล่น คืนค่า True ถ้าผู้เล่นกด Enter (ส่งคำตอบ)
        """
        submitted = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_BACKSPACE:
                self.user_input = self.user_input[:-1]
            elif event.key == pygame.K_RETURN:
                if self.user_input:  # ส่งคำตอบได้เมื่อมีข้อความเท่านั้น
                    submitted = True
            else:
                char = event.unicode
                if char.isprintable(): # รับเฉพาะตัวอักษรที่พิมพ์ได้
                    self.user_input += char
        return submitted

    def check_answer(self):
        """
        ตรวจสอบคำตอบว่าถูกต้องหรือไม่ และตั้งค่าข้อความ feedback
        """
        is_correct = self.user_input.lower() in self.valid_answers
        if is_correct:
            self.message = "Correct!"
            self.message_color = (0, 255, 0)
        else:
            self.message = "Invalid or Unknown Word!"
            self.message_color = (255, 0, 0)
        return is_correct

    def get_answer_length(self):
        """
        คืนค่าความยาวของคำตอบที่พิมพ์
        """
        return len(self.user_input)

    def clear_input(self):
        """
        ล้างช่องพิมพ์
        """
        self.user_input = ""
        self.message = "" # ล้างข้อความ feedback ด้วย

    def draw(self): # <--- สังเกตว่าตรงนี้ไม่มี (screen)
        """
        วาด UI การพิมพ์ลงบนหน้าจอ
        """
        # วาดคำถาม
        question_surf = self.font.render(f"Question: {self.question}", True, (255, 255, 255))
        self.screen.blit(question_surf, (20, 20))

        # วาดกล่องพิมพ์
        input_rect = pygame.Rect(20, 60, WIDTH - 40, 50)
        pygame.draw.rect(self.screen, (255, 255, 255), input_rect, 2) # ขอบสีขาว
        
        # วาดข้อความที่ผู้เล่นพิมพ์
        input_surf = self.font.render(self.user_input, True, (255, 255, 0))
        self.screen.blit(input_surf, (input_rect.x + 10, input_rect.y + 10))

        # วาดข้อความ feedback (Correct/Wrong)
        result_surf = self.font.render(self.message, True, self.message_color)
        self.screen.blit(result_surf, (20, 120))

# --- คลาสที่ 2: Water (จากโค้ดของคุณ) ---
WATER_RISE_PIXELS_PER_LETTER = BLOCK_SIZE  # น้ำจะขึ้นตามจำนวนตัวอักษรที่พิมพ์ได้
WATER_ANIM_DURATION = 0.9  # ระยะเวลาเคลื่อนขึ้นของน้ำ
WATER_SURFACE_WAVE_AMPLITUDE = 6
WATER_SURFACE_LENGTH = 140 # ความยาวคลื่นผิวน้ำ

class Water:
    def __init__(self, screen_h):
        self.screen_h = screen_h
        self.level_y = float(screen_h)  # ให้น้ำเริ่มจากล่างสุดจอ
        self.visible = False  # น้ำไม่แสดงตอนเริ่มเกม
 
        self.animating = False # ไม่ได้เคลื่อนที่ตอนเริ่มเกม
        self.start_y = self.level_y 
        self.target_y = self.level_y
        self.anim_time = 0.0  # เวลาเริ่มนับตอนที่น้ำแบบเริ่่มขึ้น
        self.anim_dur = WATER_ANIM_DURATION

    def start_rise(self, rise_pixels):
        if not self.visible:
            self.visible = True
        
        self.start_y = float(self.level_y)  # เป็นfloatแล้วขอบพิกเซลมันหาย
        self.target_y = max(0.0, self.start_y - rise_pixels)
        self.anim_time = 0.0
        self.animating = True

    def update(self, dt):
        if self.animating:
            self.anim_time += dt
            t = min(1.0, self.anim_time / self.anim_dur)
 
            t_ease = t * t * (3 - 2 * t) # smoothing easing ทำให้objectมันเคลื่อนแบบนุ่ม ๆ
            self.level_y = self.start_y + (self.target_y - self.start_y) * t_ease
            
            if t >= 1.0:
                self.level_y = self.target_y
                self.animating = False

    def draw(self, surf):
        if not self.visible:
            return
        
        level = int(self.level_y)
        if level < self.screen_h:
            
            # สร้าง Surface ชั่วคราวสำหรับวาดน้ำ (เพื่อให้โปร่งใสได้)
            water_surface = pygame.Surface((WIDTH, self.screen_h - level), pygame.SRCALPHA)
            water_surface.fill(WATER_COLOR)
            surf.blit(water_surface, (0, level))

            # วาดคลื่นผิวน้ำ (ส่วนนี้ทึบแสงได้)
            points = []
            t = pygame.time.get_ticks() / 1000.0
            amp = WATER_SURFACE_WAVE_AMPLITUDE
            length = WATER_SURFACE_LENGTH
            
            for x in range(0, WIDTH + 8, 8):
                y = level + math.sin((x / length) + t * 1.5) * amp
                points.append((x, y))
            
            if points:
                # วาดไฮไลท์คลื่น
                pygame.draw.lines(surf, (255, 255, 255), False, points, 3)

    def is_animating(self):
        return self.animating

    def reset(self):
        self.__init__(self.screen_h)


# --- คลาสที่ 3: Scoring (จากโค้ดของคุณ) ---
class Scoring:
    def __init__(self):
        self.char_score = 0
        self.word_score = 0
        self.words_history = []
        self.is_game_over = False

    def register_word(self, word: str):
        if self.is_game_over:
            return
        clean_word = word.strip()
        if not clean_word:
            return
        self.char_score += len(clean_word)
        self.word_score += 1
        self.words_history.append(clean_word)

    def game_over(self):
        self.is_game_over = True
        return self.get_final_score()

    def get_final_score(self):
        total = self.char_score + self.word_score
        return {
            "char_score": self.char_score,
            "word_score": self.word_score,
            "total_score": total,
            "words_history": self.words_history,
        }

    def reset(self):
        self.char_score = 0
        self.word_score = 0
        self.words_history = []
        self.is_game_over = False

# --- คลาสใหม่: Player (สำหรับจัดการบล็อกของผู้เล่น) ---
class Player:
    def __init__(self, screen_w, screen_h, block_size):
        self.screen_w = screen_w
        self.screen_h = screen_h
        self.block_size = block_size
        
        self.width = 100 # ความกว้างของบล็อก
        self.x_pos = (self.screen_w // 2) - (self.width // 2) # ตำแหน่ง X (กึ่งกลางจอ)
        
        self.color = (200, 200, 200) # สีเทา
        self.survivor_color = (255, 0, 0) # สีแดง
        
        self.total_height_pixels = self.block_size # เริ่มต้น 1 บล็อก
        self.y_pos = self.screen_h - self.total_height_pixels # ตำแหน่ง Y (บนสุดของบล็อก)

    def add_height(self, num_letters):
        """
        เพิ่มความสูงของบล็อกตามจำนวนตัวอักษร
        """
        self.total_height_pixels += num_letters * self.block_size
        self.y_pos = self.screen_h - self.total_height_pixels

    def get_top_y(self):
        """
        คืนค่าพิกัด Y ของ "หัว" ผู้เล่น (เหนือบล็อก)
        """
        return self.y_pos - 20 # 20 คือความสูงของตัวผู้เล่น

    def draw(self, surf):
        """
        วาดกองบล็อกและตัวผู้เล่น
        """
        # วาดกองบล็อก
        player_rect = pygame.Rect(self.x_pos, self.y_pos, self.width, self.total_height_pixels)
        pygame.draw.rect(surf, self.color, player_rect)
        
        # วาด "survivor" (สี่เหลี่ยมสีแดง) บนสุด
        survivor_rect = pygame.Rect(self.x_pos + (self.width // 2) - 10, self.get_top_y(), 20, 20)
        pygame.draw.rect(surf, self.survivor_color, survivor_rect)

    def reset(self):
        self.__init__(self.screen_w, self.screen_h, self.block_size)

# --- ฟังก์ชันหลัก (Main Game) ---
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Text or Die - Demo")
    
    # โหลด Font (ใช้ font เริ่มต้นของ pygame)
    try:
        # ลองใช้ font ที่ดูดีหน่อย
        main_font = pygame.font.Font(None, 40)
        score_font = pygame.font.Font(None, 50)
        small_font = pygame.font.Font(None, 24)
    except:
        # ถ้าไม่เจอก็ใช้ default
        main_font = pygame.font.SysFont(None, 40)
        score_font = pygame.font.SysFont(None, 50)
        small_font = pygame.font.SysFont(None, 24)

    clock = pygame.time.Clock()

    # --- ตั้งค่าเกม ---
    # คำศัพท์สำหรับเดโม่ (ในเกมจริงควรดึงมาจาก API หรือไฟล์ txt)
    valid_answers = [
        "apple", "banana", "orange", "grape", "mango", "melon", "kiwi",
        "pear", "peach", "plum", "cherry", "berry", "lime", "lemon"
    ]
    question = "Name a fruit"

    # สร้าง Object จากคลาสต่างๆ
    player = Player(WIDTH, HEIGHT, BLOCK_SIZE)
    water = Water(HEIGHT)
    scoring = Scoring()
    typer = typingText(screen, main_font, question, valid_answers)

    running = True
    game_over = False
    final_score_data = {}

    # --- Game Loop ---
    while running:
        # จำกัด Framerate และเอาค่า Delta Time (dt)
        dt = clock.tick(60) / 1000.0

        # --- 1. จัดการ Events ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            if game_over:
                # ถ้าเกมจบแล้ว กด 'R' เพื่อเริ่มใหม่
                if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
                    player.reset()
                    water.reset()
                    scoring.reset()
                    typer.clear_input()
                    game_over = False
                    final_score_data = {}
            else:
                # ถ้าเกมยังไม่จบ และน้ำกำลังไม่เคลื่อนไหว
                if not water.is_animating():
                    # ส่ง event ไปให้ typer จัดการ
                    submitted = typer.handle_event(event)
                    
                    if submitted:
                        is_correct = typer.check_answer()
                        player_letters = 0
                        
                        if is_correct:
                            # ถ้าตอบถูก
                            player_letters = typer.get_answer_length()
                            scoring.register_word(typer.user_input)
                            player.add_height(player_letters)
                        
                        # ไม่ว่าจะถูกหรือผิด น้ำจะขึ้น
                        # จำลองการตอบของคู่ต่อสู้ (สุ่มความยาว 3-7 ตัวอักษร)
                        opponent_len = random.randint(3, 7)
                        
                        # **ตรรกะสำคัญ:**
                        # ใน Text or Die น้ำจะขึ้นเท่ากับ "คำตอบของคนอื่น"
                        # ผู้เล่นจะรอดถ้า "บล็อก" ของตัวเองสูงกว่าน้ำ
                        
                        rise_px = opponent_len * WATER_RISE_PIXELS_PER_LETTER
                        water.start_rise(rise_px)
                        
                        # เคลียร์ช่องพิมพ์หลังจากส่งคำตอบ
                        typer.clear_input()

        # --- 2. อัปเดตสถานะเกม ---
        if not game_over:
            water.update(dt)
            
            # ตรวจสอบ Game Over
            # (เมื่อน้ำหยุดเคลื่อนที่ และ ระดับน้ำสูงกว่าหัวผู้เล่น)
            if not water.is_animating() and water.visible and water.level_y <= player.get_top_y():
                game_over = True
                final_score_data = scoring.game_over()

        # --- 3. วาดทุกอย่างลงจอ ---
        screen.fill(BACKGROUND_COLOR)
        
        # วาดผู้เล่นและบล็อก
        player.draw(screen)
        
        # วาดน้ำ
        water.draw(screen)
        
        if game_over:
            # --- วาดหน้าจอ Game Over ---
            # วาด overlay กึ่งโปร่งใส
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            
            # วาดข้อความ Game Over
            go_text = score_font.render("GAME OVER", True, (255, 0, 0))
            screen.blit(go_text, (WIDTH // 2 - go_text.get_width() // 2, HEIGHT // 3))
            
            # วาดคะแนน
            char_text = main_font.render(f"Character Score: {final_score_data.get('char_score', 0)}", True, (255, 255, 255))
            word_text = main_font.render(f"Word Score: {final_score_data.get('word_score', 0)}", True, (255, 255, 255))
            total_text = score_font.render(f"Total Score: {final_score_data.get('total_score', 0)}", True, (255, 255, 0))
            
            screen.blit(total_text, (WIDTH // 2 - total_text.get_width() // 2, HEIGHT // 2))
            screen.blit(char_text, (WIDTH // 2 - char_text.get_width() // 2, HEIGHT // 2 + 60))
            screen.blit(word_text, (WIDTH // 2 - word_text.get_width() // 2, HEIGHT // 2 + 100))
            
            # วาดคำแนะนำให้เริ่มใหม่
            restart_text = small_font.render("Press 'R' to Restart", True, (150, 150, 150))
            screen.blit(restart_text, (WIDTH // 2 - restart_text.get_width() // 2, HEIGHT - 50))
            
        else:
            # --- วาด UI การพิมพ์ ---
            typer.draw() # <--- จุดที่แก้ไข
            
        pygame.display.flip()

    pygame.quit()

# --- เริ่มเกม ---
if __name__ == "__main__":
    main()